# K-Means Clustering

Importing all necessary Libraries

In [0]:
import h5py    
import numpy as np 
import matplotlib.pyplot as plt
from pyjet import cluster,DTYPE_PTEPM
import pandas as pd
from matplotlib.pyplot import cm
from matplotlib.colors import LinearSegmentedColormap
import os
import sklearn
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import scale
import sklearn.metrics as sm
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
from sklearn.preprocessing import StandardScaler

# Reading .h5 file

In [0]:
def readHadrons(mytype, filename = 'hadron.h5'):
    savename = H5output + mytype + '_' + filename
    if not os.path.exists(savename):
        print('\x1b[31m[Warn.]\x1b[0m', savename, '\x1b[31mNOT\x1b[0m', 'found')
        return
    keys = []
    with h5py.File(savename, 'r') as f:
        keys = list(f)
    for key in keys:
        event = pd.read_hdf(savename, key=key)
        events_combined = event.T
        np.shape(events_combined)
        yield events_combined

# Elbow curve and K-Means clustering

In [0]:
H5output = '../h5files/'
for event in readHadrons('background'):
    feature = np.asarray(event)
    b = feature[[1,2],:]
    X = b.transpose()
    X1 = feature[1]
    Y1 = feature[2]
    X_std = StandardScaler().fit_transform(X)
    
    
    sse = []
    list_k = list(range(1, 10))

    for k in list_k:
        km = KMeans(n_clusters=k)
        km.fit(X_std)
        sse.append(km.inertia_)

    # Plot sse against k
    plt.figure(figsize=(4, 4))
    plt.plot(list_k, sse, '-o')
    plt.xlabel(r'Number of clusters *k*')
    plt.ylabel('Sum of squared distance')
    plt.title('Elbow curve', fontweight='bold')
    
    
    k = 5
    fig = plt.figure(figsize=(20, 4))
    for j in range(1,k+1):
        km = KMeans(n_clusters=j, max_iter=100)
        km.fit(X_std)
        centroids = km.cluster_centers_
        #print(km.labels_)
        #print(centroids)

        # Plot the clustered data
        color = ['green','blue','red','yellow','black','orange']
        for i in range(0,j):
            ax = fig.add_subplot(1, k, j)
            plt.scatter(X1[km.labels_ == i], Y1[km.labels_ == i], c=color[i], label=i+1)
            #plt.scatter(centroids[i,0], centroids[i,1], marker='*', s=300, c='r', label='centroid')
        plt.legend()
        plt.xlabel('\u03B7')
        plt.ylabel('\u03A6')
        plt.title('k = '+str(j), fontweight='bold')
        ax.set_aspect('equal');